## Preparation
### Import

In [1]:
import numpy as np
import pandas as pd
# %pip install -q amplpy gspread --upgrade
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin","highs"],  # modules to install
    license_uuid="f86e025b-7515-40af-9caf-d3bdff5ec30d",  # license to use
)  # instantiate AMPL object and register magics


Licensed to 60-day trial license.


In [2]:
%%ampl_eval
option version;

option version 'AMPL Development Version 20240606 (MSVC 19.40.33811.0, 64-bit)\
Licensed to 60-day trial license.\
Temporary license expires 20240803.\
Using license file "c:\Users\ASUS\miniconda3\envs\dunnbebes\lib\site-packages\ampl_module_base\bin\ampl.lic".\
';


## Model
### Mathematical model

In [3]:
%%writefile shipment.mod
reset;

# Sets
set SupplierID;
set FacilityID;
set ShipToID;
set ProductID;
set TransferFlows within {FacilityID, FacilityID};

# Parameters
param DemandInKG             {ShipToID,   ProductID}             >= 0 default 0;
## Facility
param StartingInventory      {FacilityID, ProductID}             >= 0 default 0;
param EndingInventory        {FacilityID, ProductID}             >= 0 default 0;
# param FacilityCapInKG        {FacilityID}                        >= 0 default 0;
# param FacilityCapInCBM       {FacilityID}                        >= 0 default 0;

# ## Conversion rate
# param CR_kg_to_CBM           {ProductID}                         >= 0 default 0;
# ## Cost
# param OrderingCost           {SupplierID, FacilityID}            >= 0 default 0;
# param TransportCost_transfer {FacilityID, FacilityID}            >= 0 default 0;
# param TransportCost_toShipTo {FacilityID, ShipToID}              >= 0 default 0;
# param RentingCost            {FacilityID}                        >= 0 default 0;
# param HandlingCost           {FacilityID}                        >= 0 default 0;
# param CustomerLoss                                               >= 0 default 0;


# Variables
var MetDemand     {ShipToID, ProductID}                    >= 0;
var UnmetDemand   {ShipToID, ProductID}                    >= 0;
var Purchases     {SupplierID, FacilityID, ProductID}      >= 0;
# var Transfers     {TransferFlows, ProductID}               >= 0; # tuple
var Transfers     {FacilityID, FacilityID, ProductID}      >= 0;
var TransferIn    {FacilityID, ProductID}                  >= 0;
var TransferOut   {FacilityID, ProductID}                  >= 0;
var Shipments     {FacilityID, ShipToID, ProductID}        >= 0;

# Objective
minimize Cost:
    sum {shipto in ShipToID, product in ProductID} UnmetDemand[shipto, product];

# Constraints
## Demand
subject to Demand_Balance {shipto in ShipToID, product in ProductID}:
    MetDemand[shipto, product] + UnmetDemand[shipto, product] = DemandInKG[shipto, product];
subject to ServeCustomer_Balance {shipto in ShipToID, product in ProductID}:
    MetDemand[shipto, product] = sum{fac in FacilityID} Shipments[fac, shipto, product];

## Flow Balancing
subject to Flow_Balance {fac in FacilityID, product in ProductID}:
    StartingInventory[fac, product] 
    + sum{sup in SupplierID} Purchases [sup, fac, product] 
    + TransferIn[fac, product]
                                                            = sum{shipto in ShipToID}    Shipments[fac, shipto, product] 
                                                            + TransferOut[fac, product] 
                                                            + EndingInventory[fac, product];
                                                            
subject to Flow_Transfer_In  {fac in FacilityID, product in ProductID}:
    TransferIn[fac, product]  = sum{(OriginFac, fac) in TransferFlows} Transfers[OriginFac, fac, product];
subject to Flow_Transfer_Out {fac in FacilityID, product in ProductID}:
    TransferOut[fac, product] = sum{(fac, DestinationFac) in TransferFlows} Transfers[fac, DestinationFac, product];
# subject to Flow_Transfer_In  {fac in FacilityID, product in ProductID}:
#     TransferIn[fac, product]  = sum{OriginFac in FacilityID} Transfers[OriginFac, fac, product];
# subject to Flow_Transfer_Out {fac in FacilityID, product in ProductID}:
#     TransferOut[fac, product] = sum{DestinationFac in FacilityID} Transfers[fac, DestinationFac, product];

## Facility Capacity
# subject to Facility_Capacity_inKG {fac in FacilityID}:
#     sum {product in ProductID} Inventory[fac, product] <= FacilityCapInKG[fac];
# subject to Facility_Capacity_inCBM {fac in FacilityID}:
#     sum {product in ProductID} Inventory[fac, product]*CR_kg_to_CBM[product] <= FacilityCapInCBM[fac];

Overwriting shipment.mod


### Load Set and Parameters

In [5]:
ampl.read('shipment.mod')

# Load data into ampl
datafile  = "small_data.xlsx"
# Demand --------------------------------------------------------
demand    = pd.read_excel(datafile, sheet_name="Demand")
ProductID = sorted(demand['ProductID'].unique().tolist())
ShipToID  = sorted(demand['ShipToID'] .unique().tolist())
demand_df = (
    demand.groupby(["ShipToID", "ProductID"])["DemandInKG"]
    .sum()
    .reset_index()
)
full_grid                = pd.MultiIndex.from_product([ShipToID, ProductID], names=["ShipToID", "ProductID"]).to_frame(index=False)
demand_df                = full_grid.merge(demand_df, on=["ShipToID", "ProductID"], how="left").fillna(0)
demand_dict              = {(row['ShipToID'], row['ProductID']): round(row['DemandInKG'],4) for index, row in demand_df.iterrows()}

ampl.set['ShipToID']     = ShipToID
ampl.set['ProductID']    = ProductID
ampl.param['DemandInKG'] = demand_dict

# Supply -------------------------------------------------------
supplier                 = pd.read_excel(datafile, sheet_name="Supply")
SupplierID               = sorted(supplier['SupplierID'].unique().tolist())
ampl.set['SupplierID']   = SupplierID

# Facility ----------------------------------------------------
facility                 = pd.read_excel(datafile, sheet_name="Facility")
FacilityID               = sorted(facility['FacilityID'] .unique().tolist())
ampl.set['FacilityID']   = FacilityID

# ampl.param['FacilityCapInKG']           = facilityCapKG_dict
# ampl.param['FacilityCapInCBM']          = facilityCapCBM_dict
# ampl.param['CR_kg_to_CBM']              = CR_kg_to_CBM_dict

# ## Cost
# ampl.param['TransportCost_transfer']    = CR_kg_to_CBM_dict
# ampl.param['TransportCost_toShipTo']    = CR_kg_to_CBM_dict
# ampl.param['RentingCost']               = CR_kg_to_CBM_dict
# ampl.param['HandlingCost']              = CR_kg_to_CBM_dict
# ampl.param['CustomerLoss']              = CR_kg_to_CBM_dict

# Lane --------------------------------------------------------
transfer_flow = pd.read_csv("TransferFlows.csv")
transfer_flow = list(transfer_flow.itertuples(index=False, name=None))
ampl.getSet('TransferFlows').setValues(transfer_flow)


### Solve

In [7]:
%%ampl_eval
option solver highs;
solve;
display MetDemand, UnmetDemand;

HiGHS 1.7.0: 

HiGHS 1.7.0: optimal solution; objective 0
0 simplex iterations
0 barrier iterations
:                  MetDemand UnmetDemand    :=
2002 '102371 - AU'      0          0
2002 '102372 - AU'     20          0
2003 '102371 - AU'     10          0
2003 '102372 - AU'     30          0
2004 '102371 - AU'      0          0
2004 '102372 - AU'     14          0
;



# Output

In [8]:
"""Demand satisfaction"""
MetDemand   = ampl.var['MetDemand']  .to_pandas()
UnmetDemand = ampl.var['UnmetDemand'].to_pandas()

result1 = MetDemand.reset_index()
result1.columns = ['ShipToID', 'ProductID','MetDemand']

result2 = UnmetDemand.reset_index()
result2.columns = ['ShipToID', 'ProductID','UnmetDemand']

Demand = pd.merge(result1, result2, on=['ShipToID', 'ProductID'], how='outer')
Demand = pd.merge(Demand, demand_df, on=['ShipToID', 'ProductID'], how='outer')

display(Demand)


,ShipToID,ProductID,MetDemand,UnmetDemand,DemandInKG
0,2002,102371 - AU,0,0,0.0
1,2002,102372 - AU,20,0,20.0
2,2003,102371 - AU,10,0,10.0
3,2003,102372 - AU,30,0,30.0
4,2004,102371 - AU,0,0,0.0
5,2004,102372 - AU,14,0,14.0


In [9]:
Purchases = ampl.var['Purchases'].to_pandas()
Purchases = Purchases.reset_index()
Purchases.columns = ['SupplierID', 'FacilityID', 'ProductID', 'PurchaseVolumeInKG']
display(Purchases)

,SupplierID,FacilityID,ProductID,PurchaseVolumeInKG
0,Supp1,Fac1,102371 - AU,10
1,Supp1,Fac1,102372 - AU,64
2,Supp1,Fac2,102371 - AU,0
3,Supp1,Fac2,102372 - AU,0
4,Supp1,Fac3,102371 - AU,0
5,Supp1,Fac3,102372 - AU,0
6,Supp2,Fac1,102371 - AU,0
7,Supp2,Fac1,102372 - AU,0
8,Supp2,Fac2,102371 - AU,0
9,Supp2,Fac2,102372 - AU,0


In [10]:
Transfers = ampl.var['Transfers'].to_pandas()
Transfers = Transfers.reset_index()
Transfers.columns = ['OriginFacID', 'DestinationFacID', 'ProductID', 'TransferVolumeInKG']
display(Transfers)

,OriginFacID,DestinationFacID,ProductID,TransferVolumeInKG
0,Fac1,Fac1,102371 - AU,0
1,Fac1,Fac1,102372 - AU,0
2,Fac1,Fac2,102371 - AU,0
3,Fac1,Fac2,102372 - AU,0
4,Fac1,Fac3,102371 - AU,0
5,Fac1,Fac3,102372 - AU,0
6,Fac2,Fac1,102371 - AU,0
7,Fac2,Fac1,102372 - AU,0
8,Fac2,Fac2,102371 - AU,0
9,Fac2,Fac2,102372 - AU,0


In [11]:
Shipments = ampl.var['Shipments'].to_pandas()
Shipments = Shipments.reset_index()
Shipments.columns = ['FacilityID', 'ShiptoID', 'ProductID', 'DispatchedVolumeInKG']
display(Shipments)

,FacilityID,ShiptoID,ProductID,DispatchedVolumeInKG
0,Fac1,2002,102371 - AU,0
1,Fac1,2002,102372 - AU,20
2,Fac1,2003,102371 - AU,10
3,Fac1,2003,102372 - AU,30
4,Fac1,2004,102371 - AU,0
5,Fac1,2004,102372 - AU,14
6,Fac2,2002,102371 - AU,0
7,Fac2,2002,102372 - AU,0
8,Fac2,2003,102371 - AU,0
9,Fac2,2003,102372 - AU,0


In [12]:
result_file      = pd.ExcelWriter('checking.xlsx', engine='xlsxwriter')
Purchases        .to_excel(result_file, sheet_name='Supply',    index=False)
Transfers        .to_excel(result_file, sheet_name='Transfers', index=False)
Shipments        .to_excel(result_file, sheet_name='Shipments', index=False)
Demand           .to_excel(result_file, sheet_name='Demand',    index=False)
result_file.close() 
